In [1]:
%load_ext autoreload
%autoreload 2
%cd '/home/xingyu/Projects/PlasticineLab'
%pwd
%matplotlib inline
import matplotlib.pyplot as plt
import taichi as ti
import numpy as np
import cv2
import os
from plb.utils.visualization_utils import save_rgb
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
target_path = './datasets/0915_GatherMove/target'
if not os.path.exists(target_path):
    os.makedirs(target_path, exist_ok=True)

np.random.seed(0)
from plb.envs.multitask_env import MultitaskPlasticineEnv
env = MultitaskPlasticineEnv(cfg_path=f'gather_move.yml', generating_cached_state=True)
taichi_env = env.taichi_env

/home/xingyu/Projects/PlasticineLab
[Taichi] mode=release
[Taichi] preparing sandbox at /tmp/taichi-1lhjnw1j
[Taichi] version 0.7.26, llvm 10.0.0, commit e37bdb5e, linux, python 3.8.5
[Taichi] Starting on arch=cuda
[Tina] version 0.1.1
[Tina] Taichi properties hacked
pimirives: num primitive: 3
Building primitive
action:
  dim: 7
  scale: (0.015, 0.0, 0.015, 0.0, 0.0, 0.1, 0.03)
collision_group: [0.0, 0.0, 0.0]
color: (0.7568, 0.6039, 0.4196)
friction: 1.0
init_gap: 0.4
init_pos: (0.7, 0.06, 0.5)
init_rot: (0.5, 0.5, -0.5, 0.5)
lower_bound: (0.0, 0.0, 0.0)
minimal_gap: 0.05
round: 0
shape: Gripper
size: (0.015, 0.09, 0.05)
upper_bound: (1.0, 1.0, 1.0)
variations: None
Building primitive
action:
  dim: 6
  scale: (0.015, 0.01, 0.015, 0.0, 0.0, 0.05)
collision_group: [0.0, 0.0, 0.0]
color: (0.7568, 0.6039, 0.4196)
friction: 50.0
init_pos: (0.7, 0.01, 0.5)
init_rot: (0.707, 0.707, 0.0, 0.0)
lower_bound: (0.0, 0.0, 0.0)
shape: Box
size: (0.07, 0.07, 0.02)
upper_bound: (1.0, 1.0, 1.0)
varia

In [18]:
target_path = './datasets/0915_GatherMove/target'
import lzma
import pickle
import numpy as np
import os
from plb.utils.visualization_utils import save_numpy_as_gif

N = 100
xs = np.linspace(0.3, 0.38, N)

def randomize_tool_state(cfg, box_min_x, pin_min_y):
    pos = eval(cfg.PRIMITIVES[0]['init_pos'])
    height = np.random.rand() * 0.1 
    x_shift = np.random.rand() * 0.1 - 0.1
    cfg.PRIMITIVES[0]['init_pos'] = (pos[0] + x_shift, float(pin_min_y + height), pos[2])


def case1(cfg): # Scattered
    cfg.SHAPES[0]['seed'] = i

def case2(cfg):
    cfg.SHAPES[0]['shape'] = 'sphere'
    del cfg.SHAPES[0]['pos_max'], cfg.SHAPES[0]['pos_min'], cfg.SHAPES[0]['seed']
    rs = np.linspace(0.04, 0.07, N//10)
    # pos = eval(cfg.SHAPES[0]['init_pos'])
    pos =  (0.65, 0.08, 0.5)
    r = rs[i * 11117771 % 12837119 % N//10]
    cfg.SHAPES[0]['radius'] = r
    new_pos = (xs[i], r + 0.1, pos[2])
    cfg.SHAPES[0]['init_pos'] = new_pos

target_id =0
target_imgs = []

def save_state():
    global target_id
    img = taichi_env.render(mode='rgb', img_size=256)
    state = taichi_env.get_state()
    state_name = os.path.join(target_path, f'state_{target_id}.xz')
    with lzma.open(state_name, 'wb') as f:
        pickle.dump(state, f, protocol=4)
    save_rgb(os.path.join(target_path, f'state_{target_id}.png'), np.array(img[:, :, :3]).astype(np.float32))
    np.save(os.path.join(target_path, f'target_{target_id}.npy'), taichi_env.simulator.get_x(0))
    target_id += 1
    target_imgs.append(img)


gifpath = os.path.join(target_path, 'vis_all.gif')
for i in range(N):
    env.reset(target_cfg_modifier=case1)
    for _ in range(10):
        env.step(np.array([0]* 13)) # Wait for dough to drop
    for j in range(25):
        action = np.array([0]* 7 + [0.] * 6)
        action[6]= ((j<15)-0.5) * 2
        env.step(action)
    save_state()
    taichi_env.primitives[0].set_state(0, [0.7, 0.06, 0.5, 0.707, 0.707, 0., 0., 0.4])
    for j in range(25):
        action = np.array([0]* 7 + [0.] * 6)
        action[6]= ((j<15)-0.5) * 2
        env.step(action)
    save_state()
target_imgs = np.array(target_imgs)
target_imgs1 = np.concatenate([target_imgs[::2, :, :, :],
                              target_imgs[1::2, :, :, :]])
target_imgs = []
for i in range(N):
    env.reset(target_cfg_modifier=case2)
    save_state()
target_imgs = np.array(target_imgs)
target_imgs = np.concatenate([target_imgs1, target_imgs])

np.save(os.path.join(target_path, 'target_imgs.npy'), np.array(target_imgs))
save_numpy_as_gif(target_imgs, gifpath)
from IPython.display import Image
Image(gifpath)

Env reset: No initial state during cache generation
Generating target, skipping setting emd
emd after reset: 0.0
======================WARNING: contact loss mask not set================
Env reset: No initial state during cache generation
Generating target, skipping setting emd
emd after reset: 0.0
======================WARNING: contact loss mask not set================
Env reset: No initial state during cache generation
Generating target, skipping setting emd
emd after reset: 0.0
======================WARNING: contact loss mask not set================
Env reset: No initial state during cache generation
Generating target, skipping setting emd
emd after reset: 0.0
======================WARNING: contact loss mask not set================
Env reset: No initial state during cache generation
Generating target, skipping setting emd
emd after reset: 0.0
======================WARNING: contact loss mask not set================


KeyboardInterrupt: 